
Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.


In [1]:
import pandas as pd

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

NOT: Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
%cd TDDI2022/

%ls

Veri İskeleti TDDI2022 klasöründe bulunan yarışma kapsamında yarışmacılara sağlanan kanunum-nlp-doc-analysis-dataset.csv dosyasının içindeki verileri pandas ile okuyarak oluşturulacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu verbose=True (yazdırılsın) parametresi ile kontrol edilecektir.

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

Tarih verilerinin formatlarının farklı olması nedeniyle hapsini aynı şekildeki bir stringe dönüştürüp karşılaştırma yapılmasını sağlamaktadır.

In [5]:


def tarihduzelt(tarih):
  ayracsayisi = 0 
  i = 0
  gun = ''
  ay = ''
  yil = ''
  if tarih.find('-') != -1: #Veri iskeletindeki tarihler "-" karakteri ile ayrılmaktadır ve yyyy-mm-dd formatında olduğundan tersten bölümleri okumayı sağlıyor.
      ayracsayisi = 2
  while i < len(tarih):
    if tarih[i].isdigit() and ayracsayisi == 0: 
      gun = gun + tarih[i]
    if tarih[i].isdigit() and ayracsayisi == 1:
      ay = ay + tarih[i]
    if tarih[i].isdigit() and ayracsayisi == 2:
      yil = yil + tarih[i]

    if tarih[i] == '/' or tarih[i] == '.':
      ayracsayisi += 1
    if tarih[i] == '-':
      ayracsayisi -= 1
    i += 1

  #gün ve ayları 2 basamağa çevirme
  if len(gun) < 2:
    gun = '0' + gun
  if len(ay) < 2:
    ay = '0' + ay
  return gun+ay+yil


In [ ]:

rg_no_dogru=0  
rg_no_yanlis=0
rg_trh_dogru=0
rg_trh_yanlis=0
bkk_no_dogru=0
bkk_no_yanlis=0
kr_trh_dogru=0
kn_trh_yanlis=0
mukerrer_dogru=0
mukerrer_yanlis=0
i = 0;
for i in range(1856,1938):  #Tüzüklerin bulunduğu aralık
  print(i)
  
  rg_trh2 = df.loc[i]['rega_tarihi']
  rg_no2 = int(df.loc[i]['rega_no'])
  bkk_no2 = df.loc[i]['mevzuat_no']
  kr_trh2 = df.loc[i]['mevzuat_tarihi']


  rg_trh2 = tarihduzelt(rg_trh2)
  kr_trh2 = tarihduzelt(kr_trh2)
  
  metin = df['data_text'][i][:310].lower()  #Verileri çıkarmak için ilk 310 karakter yeterli
  metin = metin.replace('  ','')

  #Resmi gazete tarihi çıkarma
  baslangic_rgtrh = metin.find('resmî gazete tarihi:')
  baslangic_rgtrh = baslangic_rgtrh + len('resmî gazete tarihi: ')
  rg_trh1 = metin[baslangic_rgtrh:baslangic_rgtrh+10]
  print(rg_trh1, "->", rg_trh2)

  #Resmi gazete no çıkarma
  baslangic_rgno = metin.find('resmî gazete sayısı: ')
  baslangic_rgno = baslangic_rgno + len('resmî gazete sayısı: ')
  rg_no1 = ''
  rakambuldu = 0
  #Sayının uzunluğu değiştiğinden sabit bir aralık kullanılamıyor
  while metin[baslangic_rgno].isdigit() or rakambuldu==0:
    if metin[baslangic_rgno] == ' ' or metin[baslangic_rgno] == '\n':
      rakambuldu = 0
    elif metin[baslangic_rgno].isdigit():
      rakambuldu = 1
      rg_no1 = rg_no1 + metin[baslangic_rgno]
    baslangic_rgno = baslangic_rgno + 1
  rg_no1 = int(rg_no1)
  print(rg_no1, "->", rg_no2)
  
  #Resmi gazete no ile aynı mantık sadece gördüğü '/' karakterlerini de dahil ediyor sayıya
  baslangic_bkkno = metin.find('bkk no: ')
  baslangic_bkkno = baslangic_bkkno + len('bkk no: ')
  bkk_no1 = ''
  raambuldu = 0
  while metin[baslangic_bkkno].isdigit() or rakambuldu==0 or metin[baslangic_bkkno] == '/':
    if metin[baslangic_bkkno] == ' ' or metin[baslangic_bkkno] == '\n':
      rakambuldu = 0
    elif metin[baslangic_bkkno].isdigit() or metin[baslangic_bkkno] == '/':
      rakambuldu = 1
      bkk_no1 = bkk_no1 + metin[baslangic_bkkno]
    baslangic_bkkno = baslangic_bkkno + 1
  bkk_no1 = bkk_no1
  print(bkk_no1, "->", bkk_no2)

  baslangic_krtrh = metin.find('karar tarihi: ')
  baslangic_krtrh = baslangic_krtrh + len('karar tarihi: ')
  kr_trh1 = metin[baslangic_krtrh:baslangic_krtrh+10]
  if kr_trh1[9].isdigit() != True:
    kr_trh1 = kr_trh1[:9]
  elif kr_trh1[8].isdigit() != True:
    kr_trh1 = kr_trh1[:8]
  
  kr_trh1 = kr_trh1.replace('\n', '')
  kr_trh1 = kr_trh1.replace(' ', '')
  print(kr_trh1, "->", kr_trh2)
    

  
  rg_trh1 = tarihduzelt(rg_trh1)
  kr_trh1 = tarihduzelt(kr_trh1)

    
  if rg_no1 == rg_no2:
    rg_no_dogru += 1
  else:
    rg_no_yanlis += 1
    print("*rg no* cevap:", rg_no1, " doğrusu:", rg_no2)  
  
  if rg_trh1 == rg_trh2:
    rg_trh_dogru += 1
  else:
    rg_trh_yanlis += 1
    print("*rg_tarih* cevap:", rg_trh1, " doğrusu:", rg_trh2)  
 
  if bkk_no1 == bkk_no2:
    bkk_no_dogru += 1
  else:
    bkk_no_yanlis += 1
    print("*kn_no* cevap:", bkk_no1, " doğrusu:", bkk_no2)  
  
  if kr_trh1 == kr_trh2:
    kr_trh_dogru += 1
  else:
    kn_trh_yanlis += 1
    print("*kn_trh* cevap:", kr_trh1, " doğrusu:", kr_trh2)

    
  print("d - y :: rg_trh:", rg_trh_dogru, rg_trh_yanlis,"/   rg_no:", rg_no_dogru, rg_no_yanlis,"/   bkk_no:", bkk_no_dogru, bkk_no_yanlis,"/    kr_trh:", kr_trh_dogru, kn_trh_yanlis)
  
  print("----------")

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




  